<a href="https://colab.research.google.com/github/nwang2208/Bioinformatics-Class-Projects/blob/mapping-reads/Project3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Bio

In [ ]:
from Bio import SeqIO

Upload the reads

In [ ]:
reads = {}
for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/Project3/project3b_20000_reads_without_positions.fasta', "fasta"):
    reads[seq_record.id] = str(seq_record.seq)

In [ ]:
reads

Upload example reads

In [ ]:
reads = {}
for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/Project3/project3_1000_sample_reads_without_positions.fasta', "fasta"):
    reads[seq_record.id] = str(seq_record.seq)

In [ ]:
len(reads)

In [ ]:
spectrum = {}
for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/Project3/project3_1000_sample_spectrum.fasta', "fasta"):
    spectrum[seq_record.id] = str(seq_record.seq)

In [ ]:
len(spectrum)

Make spectrum

In [ ]:
k = 20
kmer_dict = {}

for key, value in reads.items():
    for i in range(0, len(value) - k + 1):
        seq = value[i:i+k]
        if(seq in kmer_dict.keys()):
            kmer_dict[seq].append(key)
        else:
            kmer_dict[seq] = [key]


In [ ]:
#filter out rare kmers
final_kmer_dict = {}
for key, value in kmer_dict.items():
    if len(value) > 1:
        final_kmer_dict[key] = value


In [ ]:
len(final_kmer_dict)

32140

In [ ]:
final_kmer_dict

Functions for Debru

In [ ]:
import sys
sys.setrecursionlimit(1000000)

In [ ]:
def inDegree(node, graph):
    ind = 0

    for key, value in graph.items():
        if(node in value):
            ind += 1

    return(ind)

In [ ]:
def outDegree(node, graph):
    out = len(graph[node])
    return(out)

In [ ]:
def outGraph(graph):
    outgraph = {}
    for key in graph:
        outgraph[key] = len(graph[key])
    return(outgraph)

In [ ]:
def findStart(graph):
  start = next(node for node, neighbours in graph.items() if neighbours)

  for key in graph:
      if(outDegree(key, graph) - inDegree(key, graph) == 1):
          start = key

  return(start)

In [ ]:
def dfs(curr, graph, outgraph, path):
    while(curr in outgraph.keys() and outgraph[curr] != 0 ): #still edges
        next = graph[curr][outgraph[curr] - 1]
        outgraph[curr] = outgraph[curr] - 1
        dfs(next, graph, outgraph, path)
    path.insert(0, curr)

    return(path)

In [ ]:
def overlap(patterns):
    tree = {}

    for el in patterns:
        suffix = el[1:len(el)]

        for el2 in patterns:
            prefix = el2[0:len(el2) - 1]
            if(el != el2 and prefix == suffix):
                if(el in tree.keys()):
                    tree[el].append(el2)
                else:
                    tree[el] = [el2]

    return(tree)

In [ ]:
def reconstruct(kmers):
    text = ''

    for i in range(0, len(kmers)):
        if(i == len(kmers) - 1):
            text += kmers[i]
        else:
            text += kmers[i][0]

    return(text)

In [ ]:
def StringReconstruction(patterns):
    graph = overlap(patterns)
    outgraph = outGraph(graph)
    path = []
    startNode = findStart(graph)

    path = dfs(startNode, graph, outgraph, path)
    text = reconstruct(path)

    return(text)

Building Genome

In [ ]:
kmer_list = []

for key in final_kmer_dict:
    kmer_list.append(key)


In [ ]:
text = StringReconstruction(kmer_list)

In [ ]:
text

In [ ]:
text_file = open("ref.txt", "w")
n = text_file.write(text)
text_file.close()

Match the reads with the Genome